In [ ]:
from psycopg2 import connect

In [ ]:
conn = connect(host='localhost', dbname='test1')
conn

In [ ]:
with conn.cursor() as cur:
    cur.execute('select ndoc, ddate from recept limit 10')
    for row in cur:
        print(row[0],row[1])

In [ ]:
conn.close()

In [ ]:
conn = connect(host='localhost', dbname='test1')
with conn.cursor() as cur:
    cur.execute('select ndoc, ddate from recept limit 10')
    while True:
        row = cur.fetchone()
        if row is None: 
            break
        print(row[0],row[1])
conn.close()

In [ ]:
conn = connect(host='localhost', dbname='test1')
with conn.cursor() as cur:
    cur.execute('select ndoc, ddate from recept limit 10')
    while True:
        rows = cur.fetchmany(4)
        if len(rows) == 0: 
            break
        print(rows)
conn.close()

In [ ]:
import io
conn = connect(host='localhost', dbname='test1')
with conn.cursor() as cur:
    with io.StringIO() as s:
        cur.copy_to(table='region', file = s)
        print(s.getvalue())
conn.close()

In [ ]:
conn = connect(host='localhost', dbname='test1')
with conn.cursor() as cur:
    with open('rc.tsv','w') as s:
        # cur.copy_to(table='region', file = s)
        cur.copy_to(table='(select * from region where id < 1000)', file = s)
conn.close()

In [ ]:
conn = connect(host='localhost', dbname='test1')
with conn.cursor() as cur:
    with open('rc2.tsv','r') as s:
        cur.copy_from(table='region', file = s)
conn.commit()
conn.close()

In [ ]:
conn = connect(host='localhost', dbname='test1')
with conn.cursor() as cur:
    cur.execute('delete from region where id > 2')
    conn.commit() 
conn.close()

In [ ]:
conn = connect(host='localhost', dbname='test1')
with conn.cursor() as cur:
    cur.execute("insert into city(name, region) values(%s, %s)", ['Мой город', 1]) 
    conn.commit()
conn.close()

In [ ]:
%load_ext sql

In [ ]:
%sql postgresql://localhost:5432/test1

In [ ]:
%sql \l

In [ ]:
# %sql drop database my_test;
# %sql create database my_test;

In [ ]:
%sql postgresql://localhost:5432/my_test

In [ ]:
%%sql
create table my_t(id int,
 name varchar(255)                
);
insert into my_t(id,name) values (1,'test');

In [ ]:
%%sql
select * from my_t

In [ ]:
res = %sql select * from my_t
res.DataFrame()

In [ ]:
%%sql postgresql://localhost:5432/ws-dss
select * from ws_methods limit 2

In [ ]:
result = %sql select * from ws_methods limit 2
result.csv(filename='res.csv')

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost:5432/my_test')
my_df=pd.read_csv('res.csv')
my_df.to_sql('my_table1',engine,if_exists='replace')

In [ ]:
pd.read_sql('select * from ws_methods limit 2',con='postgresql://localhost:5432/ws-dss')


In [ ]:
pd.read_sql('select * from my_table1',engine)

In [ ]:
import pandas as pd
# Так не делать!
pd.read_sql('select * from ws_jobs limit 1',con='postgresql://localhost:5432/ws-dss')
pd.read_sql('select * from ws_methods limit 2',con='postgresql://localhost:5432/ws-dss')
pd.read_sql('select * from ws_models limit 3',con='postgresql://localhost:5432/ws-dss', index_col = 'id')

In [ ]:
i = 0
for chunk in pd.read_sql("SELECT * FROM ws_jobs",
       'postgresql://localhost:5432/ws-dss', chunksize=5):
    print(f"chunk {i}")
    print(chunk)
    i += 1
    if i > 3:
        break

In [ ]:
from sqlalchemy import create_engine
from pandas.io import sql
engine = create_engine('postgresql://localhost:5432/my_test')

sql.execute("INSERT INTO my_table1(id, name) VALUES(%s, %s)", engine,
            params=[(2, 'name'),(20, 'name20')])
pd.read_sql('SELECT * FROM my_table1', engine)

In [ ]:
sql.execute("INSERT INTO my_table1(id, name) VALUES(%(id)s, %(name)s)", engine,
            params={'id': 3, 'name': 'name3'})
pd.read_sql('SELECT * FROM my_table1', engine)

In [ ]:
%timeit my_df = pd.read_sql('SELECT * FROM recept', 'postgresql://localhost:5432/test1')
len(my_df)

In [ ]:
engine = create_engine('postgresql://localhost:5432/test1')
%timeit my_df = pd.read_sql('SELECT * FROM recept', engine)
my_df = pd.read_sql('SELECT * FROM recept', engine)
len(my_df)

In [ ]:
import tempfile
conn = engine.raw_connection()
def my_read(conn):
    with tempfile.TemporaryFile() as tmpfile:
        cur = conn.cursor()
        cur.copy_expert("COPY recept TO STDOUT WITH CSV HEADER", tmpfile)
        tmpfile.seek(0)
        my_df = pd.read_csv(tmpfile)
    return my_df
%timeit my_df = my_read(conn)
len(my_df)